# Games

In [37]:
import concurrent.futures
import os
import requests

In [38]:
USERNAMES = ["magnuscarlsen", "thedarkknighttrilogy", "hikaru", "wonderfultime", "danielnaroditsky"]
USERNAMES.sort()

In [39]:
def get_archives(username : str) -> list[str]:
    archives_url = f'https://api.chess.com/pub/player/{username}/games/archives'
    print('archives_url', archives_url)
    response = requests.get(archives_url)
    print('response', response.status_code, response.text)
    archives_data = response.json()
    archives = archives_data.get("archives")
    archives.reverse()
    return archives

In [40]:
def save_pgn(options : dict):
    username : str = options.get("username", "")
    archive : str = options.get("archive", "")
    paths : list[str] = archive.split("/")
    year_month : list[str] = paths[-2:]
    [year, month] = year_month
    pgn_url : str= f"{archive}/pgn"
    response : requests.Response = requests.get(pgn_url)
    pgn_text : str = response.text.strip()
    pgn_path : str = f"./{username}/games"
    pgn_exists : bool = os.path.exists(pgn_path)
    if not pgn_exists:
        os.makedirs(pgn_path)
    if pgn_text == "":
        return
    pgn_file = open(f"{pgn_path}/{year}-{month}.pgn", "w")
    pgn_file.write(pgn_text)
    pgn_file.close()
    print(year, month, 'done')

In [41]:
def save_all_pgn(username_archives : list[dict]):
    number_of_workers = len(username_archives)
    with concurrent.futures.ThreadPoolExecutor(max_workers=number_of_workers) as executor:
        executor.map(save_pgn, username_archives)

In [42]:
for username in USERNAMES:
    print("username", username)
    archives = get_archives(username)
    username_archives = []
    for archive in archives:
        username_archives.append({
            "username": username,
            "archive": archive
        })
    save_all_pgn(username_archives=username_archives)

username danielnaroditsky
archives_url https://api.chess.com/pub/player/danielnaroditsky/games/archives
response 403 <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><link href="/cdn-cgi/styles/challenges.css" rel="stylesheet"><meta http-equiv="refresh" content="375"></head><body class="no-js"><div class="main-wrapper" role="main"><div class="main-content"><noscript><div id="challenge-error-title"><div class="h2"><span class="icon-wrapper"><div class="heading-icon warning-icon"></div></span><span id="challenge-error-text">Enable JavaScript and cookies to continue</span></div></div></noscript></div></div><script>(function(){window._cf_chl_opt={cvId: '2',cZone: "api.chess.com",cType: 'managed',cNounce: '42227',cRay: '83c075ea0c0fa041',c

JSONDecodeError: Expecting value: line 1 column 1 (char 0)